# A full business solution (using Gemini API)

## Same as Day 5, but using Google Gemini instead of OpenAI

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

# Imports

If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt.

Add your Gemini API key to `.env` as `GOOGLE_API_KEY=...` (get one at https://aistudio.google.com/api-keys).

In [9]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

# Initialize Gemini client and constants

We use the **OpenAI-compatible** Gemini endpoint so the same `openai` library works with Gemini.

**If you see `RateLimitError` (429):**  
Gemini’s free tier has strict limits (per minute and per day). You can:
1. **Wait ~1 minute** and run the cell again (per-minute limit resets).
2. **Try a different model** in the cell below (e.g. `gemini-2.5-flash-lite` or `gemini-1.5-flash`) — they have separate quotas.
3. **Check usage:** [Google AI Studio](https://aistudio.google.com) or [rate limit dashboard](https://ai.dev/rate-limit).
4. **Retry logic** is added below so transient 429s wait and retry automatically.

In [10]:
import time
from openai import RateLimitError

def gemini_create_with_retry(max_retries=2, **kwargs):
    """Call gemini.chat.completions.create; on 429 wait 60s and retry up to max_retries times."""
    last_err = None
    for attempt in range(max_retries + 1):
        try:
            return gemini.chat.completions.create(**kwargs)
        except RateLimitError as e:
            last_err = e
            if attempt < max_retries:
                wait = 60
                print(f"Rate limited (429). Waiting {wait}s before retry {attempt + 1}/{max_retries}...")
                time.sleep(wait)
            else:
                raise
    raise last_err

In [11]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)
api_key = os.getenv("GOOGLE_API_KEY")

if api_key and api_key.startswith("AIza") and len(api_key) > 10:
    print("Gemini API key looks good so far")
else:
    print("There might be a problem with your GOOGLE_API_KEY? Add it to .env (see https://aistudio.google.com/api-keys)")

# Model for link selection. If you hit 429, try: gemini-2.5-flash-lite or gemini-1.5-flash
MODEL = "gemini-2.5-flash-lite"
# Model for brochure generation
BROCHURE_MODEL = "gemini-2.5-flash-lite"

gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)

Gemini API key looks good so far


In [12]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have Gemini figure out which links are relevant

Use a call to Gemini to read the links on a webpage, and respond in structured JSON.
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".
We use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding.

In [13]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [14]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [15]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [16]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = gemini_create_with_retry(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [17]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gemini-2.5-flash-lite
Found 7 relevant links


{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog posts', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'portfolio', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'social media', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'social media', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'social media',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [18]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 10 relevant links


{'links': [{'type': 'models', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'company', 'url': 'https://huggingface.co/brand'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to Gemini.

In [19]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [20]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 6 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-5
Updated
3 days ago
•
128k
•
1.21k
MiniMaxAI/MiniMax-M2.5
Updated
2 days ago
•
11.1k
•
618
openbmb/MiniCPM-SALA
Updated
5 days ago
•
2.99k
•
444
Nanbeige/Nanbeige4.1-3B
Updated
3 days ago
•
11.1k
•
428
moonshotai/Kimi-K2.5
Updated
11 days ago
•
757k
•
2.19k
Browse 2M+ models
Spaces
Running
Reachy
298
Reachy Phone Home
📱
298
Phone focus companion for Reachy Mini
Running
730
Demo Playground
⚡
730
Free platform to access multiple AI models
Running
on
A100
Featured
394
ACE-Step v1.5
🎵
394
Music Generation Foundatio

In [21]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure:

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000]  # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 5 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-5\nUpdated\n3 days ago\n•\n128k\n•\n1.21k\nMiniMaxAI/MiniMax-M2.5\nUpdated\n2 days ago\n•\n11.1k\n•\n618\nopenbmb/MiniCPM-SALA\nUpdated\n5 days ago\n•\n2.99k\n•\n444\nNanbeige/Nanbeige4.1-3B\nUpdated\n3 days ago\n•\n11.1k\n•\n429\nmoonshotai/Kimi-K2.5\nUpdated\n11 days ago\n•\n757k\n•\n2.19k\nBrowse 2M+ models\nSpaces\nRunning\nReachy\n298\nReachy Phone Home\n📱\n298\nPhone focus

In [24]:
def create_brochure(company_name, url):
    response = gemini_create_with_retry(
        model=BROCHURE_MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 13 relevant links


# Hugging Face: The AI Community Building the Future

Hugging Face is the premier platform for the machine learning community to collaborate, discover, and build the future of artificial intelligence. We provide a central hub for over 2 million models, 1 million applications, and 500,000 datasets, empowering creators and innovators across the globe.

## What We Offer:

*   **A Collaborative Platform:** Host and work on unlimited public models, datasets, and applications. Share your work, build your ML profile, and connect with a vibrant community.
*   **Vast Model & Dataset Hub:** Explore and utilize a massive collection of pre-trained models and diverse datasets covering text, image, video, audio, and even 3D modalities.
*   **AI Applications (Spaces):** Discover and run cutting-edge AI applications, from text generation and image creation to speech synthesis and more.
*   **Open-Source Stack:** Accelerate your ML development with our robust and comprehensive open-source tools and libraries.

## For Our Customers:

Whether you're a researcher, developer, or business, Hugging Face provides the tools and resources to:

*   **Accelerate AI Development:** Leverage existing models and datasets or contribute your own to speed up your projects.
*   **Innovate with Advanced AI:** Explore and implement the latest advancements in machine learning across all modalities.
*   **Enterprise Solutions:** For teams and organizations, we offer enterprise-grade security, access controls, and dedicated support to build AI securely and efficiently.

## Join the Community:

Hugging Face is more than a platform; it's a movement. We are building the future of AI together.

*   **Explore AI Apps:** Dive into a world of AI-powered applications.
*   **Browse Models:** Discover and use over 2 million cutting-edge ML models.
*   **Discover Datasets:** Access a vast repository of datasets to train and evaluate your models.

## Careers at Hugging Face:

Hugging Face is at the forefront of AI innovation, and we're looking for passionate individuals to join our mission. We foster a collaborative and dynamic culture where your contributions have a real impact. If you're excited about shaping the future of machine learning and want to work with a world-class team, explore opportunities with us.

## Finally - streaming the brochure

With a small adjustment, we can change this so that the results stream back from Gemini,
with the familiar typewriter animation.

In [26]:
def stream_brochure(company_name, url):
    stream = gemini_create_with_retry(
        model=BROCHURE_MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-2.5-flash-lite
Found 10 relevant links


# Hugging Face: The AI Community Building the Future

Hugging Face is the leading platform for the machine learning community to collaborate on models, datasets, and applications. We are building the future of AI, together.

## What We Offer:

*   **A Collaborative Platform:** Host and collaborate on an ever-growing repository of public models, datasets, and applications. With over 2 million models, 1 million applications, and 500,000 datasets, the possibilities are endless.
*   **Explore All Modalities:** Work with AI across text, image, video, audio, and even 3D.
*   **Accelerate Your ML:** Leverage the power of the HF open-source stack to move faster. We also offer paid Compute and Enterprise solutions for enhanced security, access controls, and dedicated support.
*   **Build Your Portfolio:** Share your work with the world and establish your machine learning profile.

## For Our Community:

Hugging Face is more than just a platform; it's a vibrant community. We are a place where developers, researchers, and enthusiasts come together to create, discover, and collaborate on machine learning advancements.

## For Businesses:

We provide enterprise-grade solutions designed to empower your team. With advanced security, robust access controls, and dedicated support, Hugging Face ensures your AI development is both secure and efficient.

## Careers:

Join a passionate team dedicated to democratizing AI and building the future. We are looking for talented individuals to help us grow and innovate. (Specific career opportunities are not detailed on the provided pages, but the company's focus on community and innovation suggests a dynamic work environment.)

Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

In [28]:
# stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document (using **Gemini** instead of OpenAI).

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner</a> and hoping people will teach me how it's done.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>